In [1]:
import requests

from utils.get_data import get_user, get_project, get_all_projects, get_all_users
from tqdm import tqdm
import pickle
import pandas as pd

In [7]:
with open('../assets/embedds.pickle', 'rb') as f:
    embeddings_dict = pickle.load(f)

In [8]:
embeddings_df = pd.DataFrame.from_dict(embeddings_dict).transpose()

In [9]:
embeddings_df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
",",0.165771,-0.517288,0.419655,-0.051957,-0.087263,-0.021959,0.089717,0.167934,-0.119870,-0.020433,...,-0.009044,-0.003489,0.187495,-0.144960,-0.020137,0.395344,-0.233079,0.025705,-0.408518,-0.224592
.,0.383446,0.011442,0.860179,0.129129,-0.266771,-0.042965,0.268558,0.321658,-0.243419,-0.057822,...,0.044906,0.143004,0.122937,-0.475062,0.100880,0.235484,-0.244221,-0.004340,-0.451731,0.027196
и,-0.010855,-0.695681,0.137943,0.078002,-0.112360,0.173395,0.444733,0.114993,0.025454,-0.001267,...,-0.142463,-0.062530,0.015674,-0.106922,-0.168451,0.215671,-0.326548,0.115889,-0.438935,-0.157844
в,0.072291,-0.373457,0.123495,0.219691,-0.415449,0.054333,-0.016865,0.366148,-0.209039,0.168200,...,-0.066749,0.003132,0.224285,-0.298141,-0.052664,0.423817,-0.368453,-0.214593,-0.278668,0.242669
</s>,0.804123,-0.131928,1.406710,0.330419,-0.397396,-0.131128,0.353531,0.324826,-0.467777,-0.049522,...,0.156702,0.000512,0.320597,-1.087240,-0.610243,0.274437,-0.429224,0.089688,-0.982359,0.325405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Камнеломкоцветные,0.063629,-0.003122,0.028805,0.011930,-0.035004,-0.039403,0.002906,-0.020911,-0.020743,-0.011403,...,-0.019126,0.052208,-0.038741,0.085745,-0.008106,-0.000644,0.034023,-0.000951,-0.002721,0.014279
Институтские,-0.048349,0.036216,0.011569,-0.024944,0.018396,0.042685,0.010997,0.033845,-0.034893,0.069966,...,0.022952,-0.000172,-0.003782,0.094354,-0.003228,0.052083,-0.016838,-0.009135,0.020744,0.044195
Канаршино,0.097013,0.012781,0.054817,-0.021362,-0.051439,0.007110,0.023580,0.060545,-0.021004,-0.024626,...,-0.017337,-0.006107,-0.007448,0.152516,0.019361,0.071130,0.129731,0.035641,0.001117,-0.064517
гуттаперчевого,-0.007488,-0.004338,-0.016877,-0.009949,0.048390,-0.030819,0.055507,0.023366,0.004203,-0.044218,...,-0.036519,0.015597,0.005451,0.125339,-0.010558,0.014769,0.024188,-0.013630,0.033302,0.017717


In [21]:
embeddings_df.columns = [str(x) for x in embeddings_df.columns.tolist()]

In [22]:
embeddings_df.to_parquet('embedds.parquet')

In [20]:
embeddings_df.to_parquet('embedds.parquet')

ValueError: parquet must have string column names

In [16]:
test_parquet = pd.read_parquet('embedds.parquet')

In [19]:
test_parquet.loc['В'].values

array([ 0.02952424,  0.03973465,  0.51920575,  0.2742361 , -0.56496364,
       -0.05899953,  0.15973805,  0.36940977, -0.28356385,  0.0317634 ,
        0.0545611 , -0.4064595 , -0.30602503,  0.1400957 , -0.42900386,
       -0.26042384, -0.35757917,  0.2937732 ,  0.6116437 , -0.30295795,
        0.02860113, -0.09477986,  0.12079441,  0.5180067 , -0.20671937,
       -0.1374913 , -0.19656545,  0.11080171,  0.00718931, -0.26903504,
        0.16081983, -0.15745698,  0.16667393, -0.38068062, -0.07165289,
        0.09804589,  0.06312373, -0.15158296,  0.19063157,  0.6958586 ,
        0.03368482,  0.130759  ,  0.17316923, -0.32077733, -0.0483497 ,
        0.23173176, -0.32908714, -0.07346074, -0.39619485,  0.19769314],
      dtype=float32)

In [10]:
chunk_size = int(embeddings_df.shape[0] / 40)

In [13]:
import pandas as pd

url="https://storage.yandexcloud.net/spacebear/chunk_1.csv"
c=pd.read_csv(url)

In [23]:
import pandas as pd

url="https://storage.yandexcloud.net/spacebear/embedds.parquet"
c=pd.read_parquet(url)

In [24]:
c.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
",",0.165771,-0.517288,0.419655,-0.051957,-0.087263,-0.021959,0.089717,0.167934,-0.119870,-0.020433,...,-0.009044,-0.003489,0.187495,-0.144960,-0.020137,0.395344,-0.233079,0.025705,-0.408518,-0.224592
.,0.383446,0.011442,0.860179,0.129129,-0.266771,-0.042965,0.268558,0.321658,-0.243419,-0.057822,...,0.044906,0.143004,0.122937,-0.475062,0.100880,0.235484,-0.244221,-0.004340,-0.451731,0.027196
и,-0.010855,-0.695681,0.137943,0.078002,-0.112360,0.173395,0.444733,0.114993,0.025454,-0.001267,...,-0.142463,-0.062530,0.015674,-0.106922,-0.168451,0.215671,-0.326548,0.115889,-0.438935,-0.157844
в,0.072291,-0.373457,0.123495,0.219691,-0.415449,0.054333,-0.016865,0.366148,-0.209039,0.168200,...,-0.066749,0.003132,0.224285,-0.298141,-0.052664,0.423817,-0.368453,-0.214593,-0.278668,0.242669
</s>,0.804123,-0.131928,1.406710,0.330419,-0.397396,-0.131128,0.353531,0.324826,-0.467777,-0.049522,...,0.156702,0.000512,0.320597,-1.087240,-0.610243,0.274437,-0.429224,0.089688,-0.982359,0.325405


In [ ]:
c

In [12]:
for i in tqdm(range(40)):
    selected_df = embeddings_df.iloc[i * chunk_size : (i + 1) * chunk_size]
    selected_df.to_html(f"chunk_{i}.html")

 12%|█▎        | 5/40 [05:05<35:39, 61.13s/it]


KeyboardInterrupt: 

In [44]:
chunk1 = pd.read_parquet('../assets/embedds_chunks/chunk_0.parquet')

In [48]:
chunk1.to_csv('chunk_1.csv')

In [49]:
chunk1.to_html('chunk_1.html')

In [ ]:
requests.get("https://storage.yandexcloud.net/spacebear/chunk_1.csv").content

In [3]:
df = pd.read_html(requests.get("https://storage.yandexcloud.net/spacebear/chunk_1.html").content, encoding='utf-8')

In [6]:
df[0]

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,",",0.165771,-0.517288,0.419655,-0.051957,-0.087263,-0.021959,0.089717,0.167934,-0.119870,...,-0.009044,-0.003489,0.187495,-0.144960,-0.020137,0.395344,-0.233079,0.025705,-0.408518,-0.224592
1,.,0.383446,0.011442,0.860179,0.129129,-0.266771,-0.042965,0.268558,0.321658,-0.243419,...,0.044906,0.143004,0.122937,-0.475062,0.100879,0.235484,-0.244221,-0.004340,-0.451731,0.027196
2,и,-0.010855,-0.695681,0.137943,0.078002,-0.112360,0.173395,0.444733,0.114993,0.025454,...,-0.142463,-0.062530,0.015674,-0.106922,-0.168451,0.215671,-0.326548,0.115889,-0.438935,-0.157844
3,в,0.072291,-0.373457,0.123495,0.219691,-0.415449,0.054333,-0.016865,0.366148,-0.209039,...,-0.066749,0.003132,0.224285,-0.298141,-0.052664,0.423817,-0.368453,-0.214593,-0.278668,0.242669
4,</s>,0.804123,-0.131928,1.406710,0.330419,-0.397396,-0.131128,0.353531,0.324826,-0.467777,...,0.156702,0.000512,0.320597,-1.087240,-0.610243,0.274437,-0.429223,0.089688,-0.982359,0.325405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,растущей,-0.075705,-0.043925,-0.095930,0.006661,0.003744,-0.012964,0.102350,0.037024,0.047819,...,-0.068315,0.010412,-0.032473,0.131673,0.043796,0.076792,0.002537,0.018638,-0.028279,0.114824
49996,декрет,-0.063870,-0.009739,0.098091,-0.119856,-0.130095,0.009319,-0.035605,0.054143,-0.225496,...,-0.153094,0.009577,-0.040595,0.172288,-0.048554,0.087732,0.041121,0.001047,-0.022923,0.015243
49997,Создана,-0.005917,0.149098,0.018228,0.085137,-0.087482,0.042693,0.028121,0.019666,0.076894,...,0.032102,-0.016018,0.037235,0.043960,0.075374,0.116087,-0.016759,-0.056523,-0.074507,0.174443
49998,CMOS,0.356326,-0.199862,0.113037,0.415710,0.150127,-0.371494,-0.020605,-0.154567,0.041291,...,0.076466,-0.035769,0.000955,0.317713,-0.149936,0.065129,-0.016162,-0.124973,0.073046,-0.018443
